# Import Libraries

In [53]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

%matplotlib inline
seed = 0
np.random.seed(seed)

tf.random.set_seed(seed)

os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2023.2'
os.environ['PATH'] = os.environ['XILINX_VITIS'] + '/bin:' + os.environ['PATH']

# Simulate Dataset

In [54]:
# -- Parameters --
fs         = 378e3        # ADC sampling rate [Hz]
bit_res    = 20           # 20-bit resolution
I_FS       = 1e-3         # full-scale = ±1 mA
DIG_FS     = 52000        # device maps ±1 mA → ±52 000 counts
N          = 100_000      # total samples
f_sig      = 1e3          # setting sine wave frequency to 1 kHz
n_channels = 4            # number of ADC input/output channels
t = np.arange(N) / fs     # time scale

# same sinusoidal current on each channel
I_true = I_FS * np.sin(2 * np.pi* f_sig * t).astype(np.float32)
I_true_ch = np.tile(I_true.reshape(-1,1), (1, n_channels)) # duplicates array 4x for 4 channels

# each channel's gain/offset/noise
gains   = np.array([1.05, 0.98, 1.02, 1.10], dtype=np.float32)   # caste everything as 32-bit FP
offsets = np.array([ 100, -200,   50,    0], dtype=np.float32)
noise   = np.random.normal(0, 50, size=(N, n_channels)).astype(np.float32)

# raw ADC counts = ideal-counts * gain + offset + noise
ideal_dig = (I_true_ch / I_FS) * DIG_FS # the ideal digital value (ex. if I_true_ch = 0.5mA, then (0.5 / 1) * 52000 = 26000)
X_raw = ideal_dig * gains + offsets + noise # [w0 * x0 + b0 + b1]
X_raw = np.clip(X_raw, -DIG_FS, + DIG_FS).astype(np.float32) # saturates values out of range

y_counts = ideal_dig.astype(np.float32) # current that we want to predict

# Create Training-Test Split of Data

## 80% Training | 20% Testing

In [55]:
X = X_raw
y = y_counts         

# 20% for testing, 80% for training
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Scale data for NN to converge faster

In [56]:
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)

print("Shapes:", X_train_val.shape, y_train_val.shape, X_test.shape, y_test.shape)

Shapes: (80000, 4) (80000, 4) (20000, 4) (20000, 4)


# Construct the Model

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from tensorflow.keras.layers import Activation
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

In [49]:
model = Sequential()
model.add(
    QDense(
        n_channels,
        input_shape=(n_channels,),
        name='calib_lin',
        kernel_quantizer=quantized_bits(bit_res, 0, alpha=1),
        bias_quantizer  =quantized_bits(bit_res, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(
    Activation(
        'linear',
        name='linear_out'
    )
)

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 calib_lin (QDense)          (None, 4)                 20        
                                                                 
 linear_out (Activation)     (None, 4)                 0         
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


# Don't meed to train with Sparsity

In [50]:
# you dont need to do sparsity for a model this simple

# Train the Model

In [51]:
train = True
if train:
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='mse',
        metrics=['mae']
    )

      # 3) Build your standard callbacks
    callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=1e-6,
        lr_cooldown=2,
        lr_minimum=1e-7,
        outputDir='model_1',
    )

    # 4) Fit with pruning enabled
    history = model.fit(
        X_train_val, 
        y_counts,
        batch_size=256,
        epochs=30,
        validation_split=0.2,
        shuffle=True,
        callbacks=callbacks.callbacks
    )

    model.save('model_1/calibrated_adc.h5')

else:
    from tensorflow.keras.models import load_model
    model = load_model('model_1/calibrated_adc.h5')

Epoch 1/50
237/250 [===========================>..] - ETA: 0s - loss: 1353591424.0000 - mae: 33123.7812
***callbacks***
saving losses to model_1/losses.log

Epoch 1: val_loss improved from inf to 1347736960.00000, saving model to model_1/KERAS_check_best_model.h5

Epoch 1: val_loss improved from inf to 1347736960.00000, saving model to model_1/KERAS_check_best_model_weights.h5

Epoch 1: saving model to model_1/KERAS_check_model_last.h5

Epoch 1: saving model to model_1/KERAS_check_model_last_weights.h5

***callbacks end***

250/250 [==============================] - 1s 2ms/step - loss: 1352429184.0000 - mae: 33109.5508 - val_loss: 1347736960.0000 - val_mae: 33039.3242 - lr: 1.0000e-04
Epoch 2/50
182/250 [====================>.........] - ETA: 0s - loss: 1354207360.0000 - mae: 33149.0742
***callbacks***
saving losses to model_1/losses.log

Epoch 2: val_loss did not improve from 1347736960.00000

Epoch 2: val_loss did not improve from 1347736960.00000

Epoch 2: saving model to model_1/KE

# Evaluation Metrics

In [52]:
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest MAE = {mae*1e6:.1f} µA")


Test MAE = 310889.3 µA


In [ ]:
y_pred = model.predict(X_test)

plt.figure(figsize=(5,5))
plt.scatter(y_test.flatten(), y_pred.flatten(), s=5, alpha=0.3)
lim = CODE_FS / CODE_FS * I_FS
plt.plot([-I_FS,I_FS], [-I_FS,I_FS], 'r--')
plt.xlabel("True current (A)")
plt.ylabel("Predicted current (A)")
plt.title("Calibration: Pred vs True")
plt.axis('equal')
plt.grid(True)
plt.show()

# Load Model

In [ ]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
import os

os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2019.2'
os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

model = load_model('model_1/calibrated_adc.h5', custom_objects=co)

# Convert to hls4ml model

In [ ]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['calib_lin']['ReuseFactor'] = 64

print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='model_1/hls4ml_prj_calibrated_adc', backend='VivadoAccelerator', board='pynq-z2'
)
hls_model.compile()

In [ ]:
plotting.print_dict(hls4ml.backends.get_backend('VivadoAccelerator').create_initial_config())

# CPU Simulation of hls4ml model

In [ ]:
X_test_c = np.ascontiguousarray(X_test)

# Run the HLS model in CPU (C-simulation) mode
y_hls = hls_model.predict(X_test_c)

# Save the HLS outputs for later comparison 
output_path = 'model_1/y_hls.npy'
np.save(output_path, y_hls)
print(f"HLS C-sim outputs saved to {output_path}")

# Synthesize & Make Bitfile

In [ ]:
hls_model.build(csim=False, synth=True, export=True, bitfile=True)

# Final Resource Usage

In [ ]:
!sed -n '30,45p' model_1/hls4ml_prj_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_utilization_placed.rpt